In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA

In [ ]:
import pandas as pd

# Read the data from the file into a DataFrame
bejaia = pd.read_csv('Bejaia.csv')
sidi = pd.read_csv('Sidi.csv')

# These files contain the following columns:
#   - day: the date of the observation
#   - month: the month of the observation
#   - year: the year of the observation
#   - temperature noon (temperature max) in Celsius degrees: 22 to 42
#   - RH: Relative Humidity in %: 21 to 90
#   - Ws: Wind speed in km/h: 6 to 29
#   - Rain: total day in mm: 0 to 16.8
#   - FFMC: Fine Fuel Moisture Code (FFMC) index from the FWI system: 28.6 to 92.5
#   - DMC: Duff Moisture Code (DMC) index from the FWI system: 1.1 to 65.9
#          DCDrought Code (DC) index from the FWI system: 7 to 220.4

# Print the first five rows of the DataFrame
print(bejaia.head())
print("----")
print(sidi.head())

In [ ]:
# clean the data (clean the space in rows and columns; clean the none data)
bejaia.columns = bejaia.columns.str.strip()
display(bejaia.columns)
bejaia.dropna(inplace=True)
sidi.columns = sidi.columns.str.strip()
display(sidi.columns)
sidi.dropna(inplace=True)

bejaia.Classes = bejaia.Classes.str.strip()
display(bejaia.Classes.unique())
sidi.Classes = sidi.Classes.str.strip()
display(sidi.Classes.unique())

# drop the year column
bejaia = bejaia.drop(['year'], axis = 1)
sidi = sidi.drop(['year'], axis=1)

# divide the natural factors and FWI indices
natural_bejaia = bejaia[['Temperature', 'RH', 'Ws', 'Rain']]
fwi_bejaia = bejaia[['FFMC', 'DMC', 'DC', 'ISI', 'BUI', 'FWI']]
natural_sidi = bejaia[['Temperature', 'RH', 'Ws', 'Rain']]
fwi_sidi = bejaia[['FFMC', 'DMC', 'DC', 'ISI', 'BUI', 'FWI']]

# recode the class as a dumby variable (not fire: 0, fire: 1)
replacement_mapping = {'fire': 1, 'not fire': 0}
bejaia['Classes'] = bejaia['Classes'].replace(replacement_mapping)
sidi['Classes'] = sidi['Classes'].replace(replacement_mapping)

dep_bejaia = bejaia['Classes']
dep_sidi = sidi['Classes']

In [ ]:
# get the coefficient matrix
display(bejaia.corr())
display(sidi.corr())

In [ ]:
# get the heatmap of the coefficient matrix
sns.set(rc={'figure.figsize':(12,10)})
sns.heatmap(bejaia.corr())

In [ ]:
sns.set(rc={'figure.figsize':(12, 10)})
sns.heatmap(sidi.corr())

Based on these two heatmaps we can find a general pattern:
(1) The FWI indices seem to have a strong and positive correlation with the occurence of forest fire
(2) Natural factors like rain and relative humidity seem to have a strong and negative correlation with the occurence of forest fire
(3) Natural factors, like temperature seem to have a middle and positive correlation with the occurence of forest fire.


Another pattern we can identify is that some factors have a correlation coefficient that is so close to 1, which means we may have a multicollinearity that can introduce a bias to our regression model. In this case, we can see the correlation coefficient between BUI and DMC seem to be too high. Instead of making a distinction using naked eyes, we will use algorithim to help us filter our the highly-similar variables.

In [ ]:
# separate the dependent variable
ind_bejaia = bejaia.drop('Classes',axis=1)
dep_bejaia = bejaia['Classes']
ind_sidi = sidi.drop('Classes',axis=1)
dep_sidi = sidi['Classes']

# calculate the variance inflation factor for each variable in the dataset
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

def multico_correction(X, calc_vif):
    corr_matrix = X.corr()
    high_corr_var = np.where(corr_matrix > 0.9)
    high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

# Remove one variable from each highly correlated pair
    for var1, var2 in high_corr_var:
    # Make sure that both variables are still in the DataFrame
        if var1 in X.columns and var2 in X.columns:
            if calc_vif(X.drop(var1, axis=1))['VIF'].max() < calc_vif(X.drop(var2, axis=1))['VIF'].max():
                print(f"Dropping '{var1}' due to multicollinearity")
                X = X.drop(var1, axis=1)
            else:
                print(f"Dropping '{var2}' due to multicollinearity")
                X = X.drop(var2, axis=1)
        else:
            # If one of the variables has already been dropped, print a message
            if var1 not in X.columns:
                print(f"'{var1}' has already been dropped")
            if var2 not in X.columns:
                print(f"'{var2}' has already been dropped")

    vif = calc_vif(X)
    return(vif)

# run the test for different datasets
print("all factors in bejaia is considered:")
display(multico_correction(ind_bejaia, calc_vif))

print("all factors in sidi is considered: ")
display(multico_correction(ind_sidi, calc_vif))

print("only natural factors in bejaia:")
display(multico_correction(natural_bejaia, calc_vif))

print("only natural factors in sidi: ")
display(multico_correction(natural_sidi, calc_vif))

print("only fwi indices in bejaia")
display(multico_correction(fwi_bejaia, calc_vif))

print("only fwi indices sidi")
display(multico_correction(fwi_sidi, calc_vif))

Based on the tables, we can see that for the FWI factors, the variance inflation factors of FFMC, DC AND ISI all drop to a number below 8 after deleting BUI, FWI and DMC. Therefore, we may consider BUI, FWI and DMC as factors that will trigger multicolinearity and should delete them in the following regression analysis. For natural factors, however, although the relation coefficient is low, they do have a VIF value that is much higher than 10. Such a high value maybe caused by (1) the data are dependent on each other, like rain and humidity, or (2) the variables are correlated in an unlinear way. To discover more about this part, scatter plots among natural factors are drawn for both datasets. However, the correlation between natural factors are inevitable, and cannot be manipulated. Therefore we will not delete any natural factors despite of the high VIF values.

In [ ]:
# principal component analysis on natural factors
scaler = StandardScaler()
bejaia_ns = scaler.fit_transform(natural_bejaia)
sidi_ns= scaler.fit_transform(natural_sidi)

# reduce to 2 components
pca = PCA(n_components = 3)
bejaia_pca = pca.fit_transform(bejaia_ns)
sidi_pca = pca.fit_transform(sidi_ns)
display(bejaia_pca)
display(sidi_pca)

In [ ]:
# draw scatter plots among the natural factors
def plot_scatter(df):
    num_vars = len(df.columns)
    fig, axs = plt.subplots(num_vars, num_vars, figsize=(15, 15))

    for i in range(num_vars):
        for j in range(num_vars):
            axs[i, j].scatter(df.iloc[:, i], df.iloc[:, j], alpha=0.5)
            axs[i, j].set_xlabel(df.columns[i])
            axs[i, j].set_ylabel(df.columns[j])
    plt.tight_layout()
    plt.show()
    
plot_scatter(natural_bejaia)
plot_scatter(natural_sidi)

In [ ]:
# scale the dataset first (so that all variables'scales are standardized)
scaler = StandardScaler()
bejaia_scale = scaler.fit_transform(ind_bejaia)
sidi_scale = scaler.transform(ind_sidi)

# linear regression part
def linear_regression(ind, dep):
    lin_reg = LinearRegression()
    lin_reg.fit(ind, dep)
    dep_pred = lin_reg.predict(ind)
    mae = mean_absolute_error(dep, dep_pred)
    score = r2_score(dep,dep_pred)
    return mae, score

error_bejaia, r2_bejaia = linear_regression(bejaia_scale, dep_bejaia)
error_sidi, r2_sidi = linear_regression(sidi_scale, dep_sidi)
print(error_bejaia, r2_bejaia)
print(error_sidi, r2_sidi)

In [ ]:
# now drop the redundant vairables from the dataset and conduct linear regression again
ind_bejaia.drop(['BUI'],axis = 1,inplace = True)
ind_bejaia.drop(['DMC'],axis = 1,inplace = True)
ind_sidi.drop(['BUI'],axis = 1,inplace = True)
ind_sidi.drop(['DMC'],axis = 1,inplace = True)
bejaia_scale2 = scaler.fit_transform(ind_bejaia)
sidi_scale2 = scaler.transform(ind_sidi)

error_bejaia2, r2_bejaia2 = linear_regression(bejaia_scale2, dep_bejaia)
error_sidi2, r2_sidi2 = linear_regression(sidi_scale2, dep_sidi)
print(error_bejaia2, r2_bejaia2)
print(error_sidi2, r2_sidi2)